In [23]:
import cv2

def create_tracker(tracker_type):
    if tracker_type == 'CSRT':
        return cv2.TrackerCSRT_create()
    elif tracker_type == 'MOSSE':
        return cv2.TrackerMOSSE_create()
    elif tracker_type == 'KCF':
        return cv2.TrackerKCF_create()
    else:
        raise ValueError(f"Unsupported tracker type: {tracker_type}")

tracker_type = 'KCF'

In [15]:
!pip install -U ultralytics filterpy --quiet

In [16]:
from ultralytics import YOLO
from google.colab import files
import os
import cv2
from filterpy.kalman import KalmanFilter
import numpy as np
import time

In [21]:
uploaded = files.upload()
input_path = list(uploaded.keys())[0]


Saving car1.mp4 to car1 (1).mp4


In [18]:
model = YOLO('yolov8s.pt')


In [24]:
cap = cv2.VideoCapture(input_path)
fps = 30
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_path = 'tracked_fps.mp4'
out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

frame_id = 0
names = model.names
known_ids = set()
track_history = dict()
missing_counter = dict()
last_position = dict()
kalman_filters = dict()
last_seen_frame = dict()

allowed_classes = [0, 2, 3]
MAX_MISSING = 100
MAX_SPEED = 100
MIN_OVERLAP_IOU = 0.005
MAX_INVISIBLE_FRAMES = 30

def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def init_kalman():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    kf.P *= 500
    kf.R *= 1.5
    kf.Q *= 0.003
    return kf

prev_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    now = time.time()
    time_diff = now - prev_time
    prev_time = now
    current_fps = 1 / time_diff if time_diff > 0 else 0

    results = model.track(source=frame, persist=True, verbose=False, tracker='bytetrack.yaml', imgsz=960, conf=0.01, iou=0.3)

    filtered_boxes = []
    current_ids = set()

    for box in results[0].boxes:
        if box.id is None:
            continue
        cls_id = int(box.cls.item())
        track_id = int(box.id.item())
        xyxy = box.xyxy[0].cpu().numpy().tolist()

        if cls_id in allowed_classes:
            cx = (xyxy[0] + xyxy[2]) / 2
            cy = (xyxy[1] + xyxy[3]) / 2

            if track_id not in kalman_filters:
                kalman_filters[track_id] = init_kalman()
                kalman_filters[track_id].x[:2] = np.array([[cx], [cy]])
            else:
                kalman_filters[track_id].predict()
                kalman_filters[track_id].update(np.array([[cx], [cy]]))
                cx, cy = kalman_filters[track_id].x[0][0], kalman_filters[track_id].x[1][0]

            if track_id in track_history:
                prev_cx, prev_cy = track_history[track_id]
                dx = abs(cx - prev_cx)
                dy = abs(cy - prev_cy)
                speed = (dx**2 + dy**2)**0.5
            else:
                speed = 0

            track_history[track_id] = (cx, cy)
            current_ids.add(track_id)

            if frame_id <= 50:
                known_ids.add(track_id)

            if track_id in known_ids and speed < MAX_SPEED:
                last_seen_frame[track_id] = frame_id
                if track_id in last_position:
                    iou = calculate_iou(xyxy, last_position[track_id])
                    if iou >= MIN_OVERLAP_IOU:
                        filtered_boxes.append(box)
                        last_position[track_id] = xyxy
                else:
                    last_position[track_id] = xyxy
                    filtered_boxes.append(box)

    for track_id in list(known_ids):
        if track_id not in current_ids:
            missing_counter[track_id] = missing_counter.get(track_id, 0) + 1
            if (frame_id - last_seen_frame.get(track_id, 0)) > MAX_INVISIBLE_FRAMES:
                known_ids.discard(track_id)
                kalman_filters.pop(track_id, None)
        else:
            missing_counter[track_id] = 0

    results[0].boxes = filtered_boxes
    annotated_frame = results[0].plot()

    for box in filtered_boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cls_id = int(box.cls.item())
        label = f"{names[cls_id]} ID:{int(box.id.item())}"
        cv2.putText(annotated_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    cv2.putText(annotated_frame, f"FPS: {current_fps:.2f}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    out.write(annotated_frame)
    frame_id += 1

cap.release()
out.release()


